Import Required Packages

In [1]:
#|export
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import psutil
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import wd_classes
from wd_classes import FormEntry
import importlib
importlib.reload(wd_classes)
from wd_classes import FormEntry
from openpyxl import load_workbook

Get current open chrome browser

In [192]:
#|export
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

#Before doing this launch chrome using cmd prompt command: 
#chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\Users\JoshG\Documents\Machine Learning\JobApp_Automation\ChromeProfile"

chrome_driver = "C:/Users/JoshG/AppData/Local/Microsoft/WindowsApps/chromedriver.exe"

driver = webdriver.Chrome(chrome_driver, chrome_options=chrome_options)

C:\Users\JoshG\AppData\Local\Temp\ipykernel_15612\2291859458.py:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_driver, chrome_options=chrome_options)


Error Handling

In [24]:
class stuck_on_login_page(Exception):
    print(f'page did not move past login page')
    pass

class stuck_on_create_account(Exception):
    print(f'Account not created')
    pass

class invalid_form_entry_type(Exception):
    print(f'answer type is not valid')

page did not move past login page
Account not created
answer type is not valid


Functions

In [4]:
#|export
#get login info saved in csv database
def wd_get_login_info(company_name,database="login_info.csv"):
    #open csv database
    df = pd.read_csv(database,dtype={'Password': str})
    
    #get email and password
    email = df.loc[df["Company"] == company_name, 'Email']
    email = email.values[0]
    password = df.loc[df["Company"] == company_name, 'Password']
    password = password.values[0]

    return email, password

In [5]:
#|export
#get login info saved in csv database
def write_login_info(company_name,email,password,database="login_info.csv"):
    #open csv database
    df = pd.read_csv(database)

    # add a new row to the DataFrame with the new person's name and company name
    new_row = pd.DataFrame({'Company': [f'{company_name}'], 'Email': [f'{email}'], 'Password': [f'{password}']})

    # concatenate the new DataFrame with the existing DataFrame
    df = pd.concat([df, new_row], ignore_index=False)

    # write the updated DataFrame to the existing CSV file
    df.to_csv('login_info.csv', index=False)

In [6]:
#|export
#check if this url has a workday account in database
def wd_check_account_exists(company_name, login_info="login_info.csv"):
    df = pd.read_csv(login_info,dtype={'Password': str})
    if company_name in df["Company"].values:
        return True
    else:
        return False

In [7]:
#|export
def check_page_type(page_type):
    #check to see if on page = "page_type"
    try:
        page_title_obj = driver.find_element_by_xpath(f'//h2[text()="{page_type}"]')
    except:
        return False
    #page_title_obj = driver.find_element_by_id("authViewTitle")
    page_title = page_title_obj.text
    print(page_title)
    if page_title == page_type:
        return True
    else:
        return False

In [8]:
#|export
#create new wd account
def wd_create_account(company_name, username, password):
    #get form items
    email_text_box = driver.find_element(By.XPATH, "//*[contains(text(), 'Email Address')]/following::input[@type='text'][1]")
    password_text_box = driver.find_element_by_css_selector("input[data-automation-id='password']")
    verify_password_text_box = driver.find_element_by_css_selector("input[data-automation-id='verifyPassword']")
    terms_checkbox = driver.find_element_by_css_selector("input[data-automation-id='createAccountCheckbox']")

    #fill in form
    email_text_box.clear()
    email_text_box.send_keys(email)

    #for some reason clear doesn't work with this field, so alternative is highlighting the text to clear
    password_text_box.send_keys(Keys.CONTROL,"a", Keys.DELETE)
    password_text_box.send_keys(password)
    password_text_box.send_keys(Keys.TAB)

    #See if password was accepted
    try:
        driver.find_element_by_css_selector("p[data-automation-id='inputError']")
        error_msg = driver.find_element_by_css_selector("p[data-automation-id='inputError']")
        if error_msg.is_displayed():
            print("Passsword has error")
        else:
            print("password valid, but inputerror label exists")

    except NoSuchElementException:
        print("Password is valid")

    verify_password_text_box.clear()
    verify_password_text_box.send_keys(password)

    #check terms and conditions checkbox
    if terms_checkbox.get_attribute("checked") == "true":
        print('was already checked')
    else:
        terms_checkbox.click()

    #Get create account button
    create_account_button = driver.find_element_by_xpath("//div[@aria-label='Create Account']")
    driver.implicitly_wait(1)
    driver.execute_script("arguments[0].click();", create_account_button)

    #Add to database if account created successfully
    if check_page_type('Create Account'):
        raise stuck_on_create_account("Did not move on to next page from create account")
    else:
        write_login_info(company_name, username, password)

In [9]:
#|export
def wd_sign_in(email, password):
    # Get email and password elements
    email_text_box = driver.find_element(By.XPATH, "//*[contains(text(), 'Email Address')]/following::input[@type='text'][1]")
    password_text_box = driver.find_element(By.XPATH, "//*[contains(text(), 'password')]/following::input[@type='password'][1]")
    
    
    #Fill in email and password
    email_text_box.send_keys(email)
    password_text_box.send_keys(password)

    #wait for a moment
    driver.implicitly_wait(2)
    
    #Get sign in button and click
    sign_in_button = driver.find_element_by_xpath("//div[@aria-label='Sign In']")
    driver.implicitly_wait(1)
    driver.execute_script("arguments[0].click();", sign_in_button)

    driver.implicitly_wait(1)

    
    #check to see if still on same page
    if check_page_type('login'):
        raise stuck_on_login_page("Login Info Didn't Work")
    else:
        print("no errors found on login page")

In [10]:
#|export
def get_current_company():
    #get url and pull the text after the first / after myworkdayjobs.com
    current_url = driver.current_url

    # Split the URL by "/"
    parts = current_url.split("/")

    # Get the text following ".com/" and before the next slash
    company_name = parts[3]

    return company_name

In [11]:
#|export
def wd_login_page(new_email,new_password):
    
    wait = WebDriverWait(driver, 20)
    wait.until(EC.presence_of_element_located((By.ID, "input-4")))
    
    current_company = get_current_company()

    #check if account exists for this site and login if has account or create account and then login
    if wd_check_account_exists(current_company) is True:
        email, password = wd_get_login_info(current_company)
        wd_sign_in(email, password)
    else:
        create_account_button = driver.find_element_by_xpath("//button[contains(text(), 'Create Account')]")
        create_account_button.click()
        
        wd_create_account(current_company, new_email, new_password)

In [12]:
#|export
def wd_startYourApplication_page():
    
    wait = WebDriverWait(driver, 20)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "[data-automation-id='applyManually']")))

    manual_apply_btn = driver.find_element_by_css_selector("a[data-automation-id='applyManually']")
    manual_apply_btn.click()

In [28]:
#|export
def get_answer(df,question):
    #pull answer from form
    try:
        index = df.index[df['Questions'] == question].tolist()[0]
        answer = df.at[index, 'Answers']
    except:
        answer = ""
        print(f"failed to get answer for: {question}")

    if pd.isna(answer):
        answer = ""

    return answer

In [185]:
#|export
def wait_data_automation_id_loaded(dataautomationid,t=10):
    wait = WebDriverWait(driver, t)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f"[data-automation-id='{dataautomationid}']")))
    time.sleep(1)

In [166]:
def fill_sub_section(driver, df, exp_container):
    # locate all input elements in work experience 1
    inputs_list = exp_container.find_elements_by_css_selector("[id^='input-']")

    #remove any elements that don't have their id ending in a number
    new_inputs_list = []
    for i in range(len(inputs_list)):
        current_id = inputs_list[i].get_attribute('ID')
        if current_id[-1].isdigit() != False:
            new_inputs_list.append(inputs_list[i])
    inputs_list = new_inputs_list

    #move through each form entry and fill in
    entries = []
    for element in inputs_list:
        #make sure element still exists
        try:
            if len(driver.find_elements_by_xpath(f"//*[@id='{element.get_attribute('ID')}']")) > 0:
                pass
        except:
            continue
        
        #create element object
        entry = FormEntry(element)
        
        #get the type of answer form it is
        entry.get_id()
        entry.get_tag()
        entry.get_answer_type()
        entry.get_label()

        #skip if a span element
        if entry.tag == 'span':
            continue

        #pull answer from form
        answer = get_answer(df,entry.label)
        entry.add_answer(answer)

        #fill in answer
        entry.update_element(driver)
        entry.write_answer(driver)
            
        #append entry to entries list
        entries.append(entry)

In [124]:
#|export
def delete_sub_section(aria_label):
    delete_btn = driver.find_elements_by_xpath(f"//button[starts-with(@aria-label,{aria_label})]")
    if len(delete_btn) > 0:
        for i in range(len(delete_btn)):
            delete_btn[i].click()

def add_sub_section(form_container_id,positioning='following-sibling'):
    add_btn = driver.find_elements_by_xpath(f"//*[@data-automation-id='{form_container_id}']/{positioning}::div//button[starts-with(@aria-label,'Add')]")
    add_btn[0].click()

In [120]:
#|export
def fill_section(browser_driver, container_auto_id, start_of_sheets_name, numb_sheets=1, qa_database="Answers.xlsx", implicit_wait_time=0.2):

    #wait until page has loaded form
    if numb_sheets == 1:
        wait_data_automation_id_loaded(container_auto_id)
    else:
        wait_data_automation_id_loaded(f"{container_auto_id}{1}")

    #get question/answer database and sheets
    sheet_names = pd.ExcelFile(qa_database).sheet_names
    sheets = [name for name in sheet_names if name.startswith(start_of_sheets_name)]
    required_sheets = list(sheets)

    #fill in each form for each job
    for i in range(numb_sheets):

        #put sheet required to fill in form as the active dataframe
        df = pd.read_excel(qa_database,required_sheets[i],dtype=str)

        #get the container element
        if numb_sheets == 1:
            form_container = driver.find_element_by_xpath(f"//div[@data-automation-id='{container_auto_id}']")
        else:
            form_container = driver.find_element_by_xpath(f"//div[@data-automation-id='{container_auto_id}{(i+1)}']")
        
        #fill in work experience section
        fill_sub_section(driver,df,form_container)

        #click on add another so long as not last work experience to add
        if i+1 != numb_sheets and numb_sheets > 1:  
            #click work experience button
            form_container_id = form_container.get_attribute('data-automation-id')
            add_sub_section(form_container_id)

In [127]:
#|export
def upload_file(file_path):
    # Locate the upload button using its ID or CSS selector
    upload_button = driver.find_element_by_xpath(f"//input[@data-automation-id='file-upload-input-ref']")
    upload_button.send_keys(file_path)


In [74]:
#|export
#
#click the save and continue button
def go_to_next():
    continue_btn = driver.find_element_by_xpath("//button[@data-automation-id='bottom-navigation-next-button']")
    continue_btn.click()


In [159]:
import wd_classes
from wd_classes import FormEntry
import importlib
importlib.reload(wd_classes)
from wd_classes import FormEntry

In [14]:
#Example job url
url = 'https://jci.wd5.myworkdayjobs.com/JCI/login?redirect=%2FJCI%2Fjob%2FMissoula-Montana-United-States-of-America%2FHVAC-Truck-Base-Mechanic--Union-_WD30167702%2Fapply%3Fsource%3DCareers_Section'

#login info
email = "pyEngineer69@outlook.com"
password = "aTestPass@1234"

#Open job link, sign in, and start manual application
def start_job_app(url, email, password):
    #get job posting url
    driver.get(url)

    #login or create account and log in
    wd_login_page(email, password)

    #Click on start manual application
    try:
        wd_startYourApplication_page()
    except:
        print("Apply manually button was not pressed. This could be because the job app had already been started or an error")

start_job_app(url,email,password)

no errors found on login page


In [77]:
page_auto_id = 'contactInformationPage'
answers_database = 'Answers.xlsx'
answer_sheet_name = 'Answers'

fill_section(driver, page_auto_id,start_of_sheets_name=answer_sheet_name,qa_database=answers_database)

go_to_next()

Have you ever worked for JCI or one of its affiliates?
failed to get answer for: 
Failed to get a label, ID: input-631


In [125]:
add_sub_section('workExperienceSection',positioning='descendant')

fill_section(driver,'workExperience-',start_of_sheets_name='WorkExp',qa_database=answers_database,numb_sheets=2)

date to input: 1/3/2023, ID: input-1325
12023
date to input: 02/18/2021, ID: input-1468
022021
date to input: 06/28/2022, ID: input-1471
062022


In [126]:
add_sub_section("educationSection",positioning="descendant")

fill_section(driver,'education-',start_of_sheets_name='Education',qa_database=answers_database,numb_sheets=2)

2019
2022
2016
2021


In [129]:
file_path = "C:\\Users\\JoshG\\Documents\\Machine Learning\\JobApp_Automation\\Sample Resume.docx"
upload_file(file_path)

In [130]:
go_to_next()

In [133]:
fill_section(driver,'primaryQuestionnairePage',start_of_sheets_name='Answers',qa_database=answers_database,numb_sheets=1)

go_to_next()

In [151]:
df = pd.read_excel('Answers.xlsx',sheet_name="RacistQuestions", dtype=str)

section = driver.find_element_by_xpath("//div[@data-automation-id='voluntaryDisclosuresPage']")
fill_sub_section(driver,df,section)

In [149]:
go_to_next()

In [183]:
fill_section(driver,'selfIdentificationPage',start_of_sheets_name='Disabilities',qa_database=answers_database,numb_sheets=1)


ID: input-2152, tag: button, entry label: Language, answer: English, answer type: LISTBOX
Failed to get a label, ID: input-2153
Failed to get a label, ID: input-2154
Failed to get a label, ID: input-2155
ID: input-2156, tag: input, entry label: Name, answer: John Smith, answer type: TEXTINPUT
ID: input-2157, tag: input, entry label: Employee ID (if applicable), answer: , answer type: TEXTINPUT
ID: input-2160, tag: div, entry label: Date, answer: 02/01/2023, answer type: DATE_MONTH_DAY_YEAR
ID: input-2161, tag: div, entry label: Please check one of the boxes below:, answer: No, I Don't Have A Disability, Or A History/Record Of Having A Disability, answer type: DISABILITY


In [193]:
#Example job url
url = "https://cardinalhealth.wd1.myworkdayjobs.com/EXT/login?redirect=%2FEXT%2Fjob%2FUS-Nationwide-FIELD%2FPrincipal-Design-Quality-Engineer_20116986%2Fapply%3Fpk_vid%3Da04039fd660829c016753066470bea81"

#login info
email = "pyEngineer69@outlook.com"
password = "aTestPass@1234"

#Open job link, sign in, and start manual application
def start_job_app(url, email, password):
    #get job posting url
    driver.get(url)

    #login or create account and log in
    wd_login_page(email, password)

    #Click on start manual application
    try:
        wd_startYourApplication_page()
    except:
        print("Apply manually button was not pressed. This could be because the job app had already been started or an error")

start_job_app(url,email,password)



Password is valid
Create Account


stuck_on_create_account: Did not move on to next page from create account

In [194]:
page_auto_id = 'contactInformationPage'
answers_database = 'Answers.xlsx'
answer_sheet_name = 'Answers'

fill_section(driver, page_auto_id,start_of_sheets_name=answer_sheet_name,qa_database=answers_database)

go_to_next()

wait_data_automation_id_loaded('workExperienceSection')
add_sub_section('workExperienceSection',positioning='descendant')

fill_section(driver,'workExperience-',start_of_sheets_name='WorkExp',qa_database=answers_database,numb_sheets=2)

add_sub_section("educationSection",positioning="descendant")

fill_section(driver,'education-',start_of_sheets_name='Education',qa_database=answers_database,numb_sheets=2)

file_path = "C:\\Users\\JoshG\\Documents\\Machine Learning\\JobApp_Automation\\Sample Resume.docx"
upload_file(file_path)

go_to_next()

fill_section(driver,'primaryQuestionnairePage',start_of_sheets_name='Answers',qa_database=answers_database,numb_sheets=1)

go_to_next()

fill_section(driver,'voluntaryDisclosuresPage',start_of_sheets_name='RacistQuestions',qa_database=answers_database,numb_sheets=1)

go_to_next()

fill_section(driver,'selfIdentificationPage',start_of_sheets_name='Disabilities',qa_database=answers_database,numb_sheets=1)

ID: input-1, tag: button, entry label: How Did You Hear About Us?, answer: LinkedIn, Other, OTR, answer type: LISTBOX
Have you previously worked for Cardinal Health? If Yes, please answer the questions below. If No, please continue to the next page
failed to get answer for: Have you previously worked for Cardinal Health? If Yes, please answer the questions below. If No, please continue to the next page
ID: input-2, tag: div, entry label: Have you previously worked for Cardinal Health? If Yes, please answer the questions below. If No, please continue to the next page, answer: , answer type: None
failed to get answer for: Country
ID: input-3, tag: button, entry label: Country, answer: , answer type: LISTBOX


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=109.0.5414.120)
